<a href="https://colab.research.google.com/github/MAfarrag/Hapi/blob/master/Examples/Hydrological%20model/Note%20books/Jiboa_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Jiboa Case Study

This code is prepared to Run the distributed model for jiboa rover in El Salvador
wher the catchment is consisted os a ustream lake and a volcanic area
-   you have to make the root directory to the examples folder to enable the code
    from reading input files

In [ ]:
if (len(conda_path) == 0):
    print('installing miniconda')
    !wget https://repo.continuum.io/miniconda/Miniconda3-4.5.4-Linux-x86_64.sh && bash Miniconda3-4.5.4-Linux-x86_64.sh -bfp /usr/local
    !conda update conda -y -q
    !source /usr/local/etc/profile.d/conda.sh
    !conda init 
    !conda install -n root _license -y -q
else:
    print('found miniconda')

In [4]:
!pip install Hapi-Nile

     |████████████████████████████████| 16.8 MB 210 kB/s 


/bin/bash: conda: command not found


In [2]:
# try to run the bare minimum to get a new conda env working
conda_path = ''
try:
    conda_path = !which conda
finally:
    print('Conda does not exist')

Conda does not exist


In [ ]:
conda_envs = !conda env list
res = [i for i in conda_envs if 'test36' in i]
if (len(res) == 0):
    print('not found test36 env', len(res))
    !conda create -y -q --name test36 python=3.6
else:
    print('found test36 env', len(res))

not found test36 env 0

InvalidVersionSpec: Invalid version '4.19.112+': empty version component



In [ ]:
!conda install --channel conda-forge featuretools --yes


/bin/bash: conda: command not found


In [9]:
pip install git+https://github.com/MAfarrag/HAPI
!pip install HAPI-Nile

  Cloning https://github.com/MAfarrag/HAPI to /tmp/pip-req-build-rsj6rc_e
  Running command git clone -q https://github.com/MAfarrag/HAPI /tmp/pip-req-build-rsj6rc_e


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


     |████████████████████████████████| 19.1 MB 52.3 MB/s 
     |████████████████████████████████| 67 kB 3.9 MB/s 


### Download Data
For the data of this case study you have to download this folder [Jiboa Data](https://drive.google.com/drive/folders/1uyd9mH8pHHUKK9l3bc7QXlsu4EPwy3Mr?usp=sharing) from Google Drive and set it as the working directory instead of the Path defined in the next cell

In [13]:
import os
Path = "/content/drive/My Drive/coello"

os.chdir(Path)

  Using cached GDAL-3.2.2.tar.gz (601 kB)
  ERROR: Failed building wheel for gdal
  Running setup.py clean for gdal
Failed to build gdal
    Running setup.py install for gdal ... error
ERROR: Command errored out with exit status 1: /usr/local/bin/python -u -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'/tmp/pip-install-pom0l1f8/gdal_b73f3e6dc916445082ed23e78d2303aa/setup.py'"'"'; __file__='"'"'/tmp/pip-install-pom0l1f8/gdal_b73f3e6dc916445082ed23e78d2303aa/setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' install --record /tmp/pip-record-g0fu_ezh/install-record.txt --single-version-externally-managed --compile --install-headers /usr/local/include/python3.7m/gdal Check the logs for full command output.
  Using cached proj-0.2.0-py2.py3-none-any.whl (8.6 kB)
     |████████████████████████████████| 54 kB 1.7 MB/s 
ERROR: Could not find a version that satis

  Created wheel for ecmwf-api-client: filename=ecmwf_api_client-1.6.1-py3-none-any.whl size=12619 sha256=0c9bf01eed4ecad40a1eb9c1c94521e903dccf4e1da4a7db31fc23f9af207a9a
  Stored in directory: /root/.cache/pip/wheels/bd/6b/ce/7dfe55ed168bf71dd725f3c452fc3222767f09459410633982
Successfully built ecmwf-api-client


In [19]:
! pip install Hapi-Nile

In [21]:
! 

ModuleNotFoundError: ignored

In [4]:
os.listdir()

['Data', 'Model', 'requirement.txt']

## Import modules

In [17]:
import os
Path = "/content/drive/My Drive/coello"

os.chdir(Path)

#%library
import gdal
import datetime as dt
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# HAPI modules
from Hapi.run import RunHAPIwithLake
import Hapi.hbv as HBV
import Hapi.performancecriteria as Pf
import Hapi.raster as Raster

ModuleNotFoundError: ignored

In [ ]:
# the beginning of the simulation of the calibration data
start = dt.datetime(2012,6,14,19,00,00)
end = dt.datetime(2014,11,17,00,00,00)
calib_end = dt.datetime(2013,12,23,00,00,00)

# paths to the meteorological data
PrecPath = prec_path = "inputs/Hapi/meteodata/4000/calib/prec_clipped"
Evap_Path = evap_path = "inputs/Hapi/meteodata/4000/calib/evap_clipped"
TempPath = temp_path = "inputs/Hapi/meteodata/4000/calib/temp_clipped"

#DemPath = path+"GIS/4000/dem4000.tif"
FlowAccPath = "inputs/Hapi/GIS/4000_matched/acc4000.tif"
FlowDPath = "inputs/Hapi/GIS/4000_matched/fd4000.tif"
ParPath = "inputs/Hapi/meteodata/4000/parameters/"
#ParPath = "inputs/Hapi/meteodata/4000/"+"parameters.txt"
Paths=[PrecPath, Evap_Path, TempPath, FlowAccPath, FlowDPath, ]

In [ ]:

#p2=[24, 1530]
#init_st=[0,5,5,5,0]
init_st = np.loadtxt("inputs/Hapi/meteodata/Initia-jiboa.txt", usecols=0).tolist()
snow = 0


# lake meteorological data
ind = pd.date_range(start, end, freq = "H" )
lakedata = pd.read_csv("inputs/Hapi/meteodata/lakedata.csv", index_col = 0)
lakedata.index = ind
lakeCalib = lakedata.loc[start:calib_end]
lakeValid = lakedata.loc[calib_end:end]
# convert the dataframe into array
lakeCalibArray = lakeCalib.values
# take only the plake, et, t and tm columns and exclude the last column
lakeCalibArray = lakeCalibArray[:,0:-1]

# where the lake discharges its flow (give the indices of the cell)
lakecell = [2,1]    # 4km
#lakecell = [4,2]    # 2km
#lakecell = [10,4]    # 1km
#lakecell = [19,10]    # 500m

LakeParameters = np.loadtxt("inputs/Hapi/meteodata/4000/Lakeparameters.txt").tolist()
StageDischargeCurve = np.loadtxt("inputs/Hapi/meteodata/curve.txt")
p2 = [1, 227.31, 133.98, 70.64]
Lake_init_st = np.loadtxt("inputs/Hapi/meteodata/Initia-lake.txt", usecols=0).tolist()

# Run the model

In [ ]:
os.getcwd()

'/content/drive/My Drive'

In [ ]:
os.listdir()

['Hapi', 'Jiboa.mxd', 'web application', 'processing', 'inputs', 'results.zip']

In [ ]:
Sim =pd.DataFrame(index = lakeCalib.index)
st, Sim['Q'], q_uz_routed, q_lz_trans = RunHAPIwithLake(HBV, Paths, ParPath, p2, init_st,
                                                     snow, lakeCalibArray, StageDischargeCurve,
                                                     LakeParameters, lakecell,Lake_init_st)

### Evaluate model performance

In [ ]:
WS = {}
WS['type'] = 1
WS['N'] = 3
ModelMetrics=dict()
ModelMetrics['CalibErrorHf']=Pf.RMSEHF(lakeCalib['Q'],Sim['Q'],WS['type'],WS['N'],0.75)
ModelMetrics['CalibErrorLf']=Pf.RMSELF(lakeCalib['Q'],Sim['Q'],WS['type'],WS['N'],0.75)
ModelMetrics['CalibNSEHf']=Pf.NSE(lakeCalib['Q'],Sim['Q'])
ModelMetrics['CalibNSELf']=Pf.NSE(np.log(lakeCalib['Q']),np.log(Sim['Q']))
ModelMetrics['CalibRMSE']=Pf.RMSE(lakeCalib['Q'],Sim['Q'])
ModelMetrics['CalibKGE']=Pf.KGE(lakeCalib['Q'],Sim['Q'])
ModelMetrics['CalibWB']=Pf.WB(lakeCalib['Q'],Sim['Q'])
print("RMSE(HF) = " + str(round(ModelMetrics['CalibErrorHf'],2)))
print("RMSE(LF) = " + str(round(ModelMetrics['CalibErrorLf'],2)))
print("RMSE = " + str(round(ModelMetrics['CalibRMSE'],2)))
print("NSE(HF) = " + str(round(ModelMetrics['CalibNSEHf'],2)))
print("NSE(LF) = " + str(round(ModelMetrics['CalibNSELf'],2)))
print("KGE = " + str(round(ModelMetrics['CalibKGE'],2)))
print("WB = " + str(round(ModelMetrics['CalibWB'],2)))

In [ ]:
plt.figure(50,figsize=(15,8))
Sim.Q.plot(color=[(0,0.3,0.7)],linewidth=2.5,label="Simulated data", zorder = 10)
ax1=lakeCalib['Q'].plot(color='#DC143C',linewidth=2.8,label='Observed data')
ax1.annotate("Model performance" ,xy=('2012-12-01 00:00:00',20),fontsize=15)
ax1.annotate("RMSE = " + str(round(ModelMetrics['CalibRMSE'],3)),xy=('2012-12-01 00:00:00',20-1.5),fontsize=15)
ax1.annotate("NSE = " + str(round(ModelMetrics['CalibNSEHf'],2)),xy=('2012-12-01 00:00:00',20-3),fontsize=15)
plt.legend()

# Store the result into rasters

In [ ]:
# create list of names
src=gdal.Open(FlowAccPath)

index=pd.date_range(start,calib_end,freq="1H")

resultspath="results/upper_zone_discharge/4000/"
names=[resultspath+str(i)[:-6] for i in index]
names=[i.replace("-","_") for i in names]
names=[i.replace(" ","_") for i in names]
names=[i+".tif" for i in names]


Raster.RastersLike(src,q_uz_routed[:,:,:-1],names)
